In [ ]:
import numpy as np
import pandas as pd
import string
import os
import seaborn as sns
import matplotlib.pyplot as plt
import json

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from wordcloud import STOPWORDS
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow import keras

from sklearn.model_selection import train_test_split,KFold

import warnings
warnings.filterwarnings("ignore")

In [ ]:
sample = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
df_target = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')
target =  pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
# Remove stopwords & convert to lower case
df_target['text'] = df_target['text'].apply(lambda x: ' '.join([w for w in str(x).lower().split() if w not in STOPWORDS]))

# Remove Punctuations
df_target["text"] = df_target['text'].str.replace('[^\w\s]','')
df_target.head()
MAX_LEN = 100
preds = []
saved_paths = ['../input/lstmweights/lstm_0',
         '../input/lstmweights/lstm_1',
         '../input/lstmweights/lstm_2',
         '../input/lstmweights/lstm_3',
         '../input/lstmweights/lstm_4',
        
        ]
for path in saved_paths:
    with open(path+'/tokenizer.json') as f:
        data = json.load(f)
        tokenizer = tokenizer_from_json(data)
    target_seq = tokenizer.texts_to_sequences(df_target.text.values)
    target_padded = pad_sequences(
        target_seq, maxlen=MAX_LEN, dtype='int32', padding='post',
        truncating='post'
    )
    
    model = keras.models.load_model(path)
    result = model.predict(target_padded)
    preds.append(result)


In [ ]:
preds = np.array(preds)
preds= np.mean(preds, axis=0)
target['score'] = preds
target[['comment_id','score']].to_csv('./submission.csv', index=False)